In [1]:
from datetime import datetime
import sys
from pprint import pformat
import random

import pandas as pd

sys.path.append("/workspace/multivariate-correlation-anomaly-detection/")
from utils.assorted_utils import load_multiple_data, load_dirs, load_data_cfg
from utils.select_items_utils import (gen_random_items, gen_corr_prop_filtered_items, gen_pca_cluster_filtered_samples_two_max_dist_clusters,
                                      gen_pca_cluster_filtered_samples_each_cluster, gen_pca_cluster_samples_with_given_sample)
from utils.cluster_utils import (calc_pca, obs_various_n_clusters_hrchy_cluster)
from utils.log_utils import Log

JPY_LOGGER = Log(df_max_rows=100, df_max_columns=10).init_logger(logger_name="ywt_jupyter")

time: 534 ms (started: 2023-12-19 22:36:28 +08:00)


# Prepare data

## Load Data

In [2]:
data_implement = "SP500_20112015"  # watch options by printing /config/data_config.yaml/["DATASETS"].keys()
# etl set setting
retrieve_items_setting = "-train_train"  # -train_train|-train_all
# set correlation type
corr_type = "pearson"  # "pearson" | "cross_corr"
# set target_df bins
target_df_bins = "bins_-10_-03_03_10"
# set CORR_WINDOW and CORR_STRIDE length
w_l=50 ; s_l = 1
# Decide how to calculate corr_ser
corr_ser_clac_method = "corr_ser_calc_regular"  # corr_ser_calc_regular|corr_ser_calc_abs

time: 403 µs (started: 2023-12-19 22:36:35 +08:00)


In [3]:
pipeline_corr_data_dir, corr_dir, target_dir, corr_property_dir, cliques_dir, cluster_dir = load_dirs(data_implement=data_implement,
                                                                                                      retrieve_items_setting=retrieve_items_setting,
                                                                                                      corr_type=corr_type, target_df_bins=target_df_bins,
                                                                                                      w_l=w_l, s_l=s_l,
                                                                                                      corr_ser_clac_method=corr_ser_clac_method)
dataset_df, corr_df, target_df, corr_property_df = load_multiple_data(data_implement=data_implement,
                                                                      retrieve_items_setting=retrieve_items_setting,
                                                                      corr_type=corr_type, target_df_bins=target_df_bins,
                                                                      w_l=w_l, s_l=s_l,
                                                                      corr_ser_clac_method=corr_ser_clac_method)
display(dataset_df)
display(corr_df)
display(target_df)
display(corr_property_df)

INFO     [utils.etl_utils.calc_corr_ser_property]  :  
corr_property_df exists, corr_property_df loaded from /workspace/multivariate-correlation-anomaly-detection/dataset/pipeline_dataset/sp500_20112015-train_train/pearson/corr_property/corr_s1_w50/corr_ser_calc_regular/corr_series_property.csv
INFO     [utils.assorted_utils.load_multiple_data] =========================================== Info of Datasets ===========================================
INFO     [utils.assorted_utils.load_multiple_data] len(items_implement): 100 and len(all_set): 389 and len(train_set): 100
INFO     [utils.assorted_utils.load_multiple_data] output_file_name: sp500_20112015-train_train, corr_s1_w50 and corr_ser_clac_method:corr_ser_calc_regular
INFO     [utils.assorted_utils.load_multiple_data] dataset_df.shape:(1258, 100), corr_df.shape:(4950, 1209), target_df.shape:(4950, 1209)
INFO     [utils.assorted_utils.load_multiple_data] corr_property_df.shape: (4950, 6)
INFO     [utils.assorted_utils.load_multiple_d

,AAP,ADBE,AFL,AIG,AIV,...,UNP,UPS,VTR,VZ,ZBH
Date,,,,,,,,,,,
2011-01-03,59.302174,31.290001,20.970913,38.270241,1.912387,...,35.566772,49.716789,34.297752,19.474819,48.349262
2011-01-04,56.350639,31.510000,20.588228,37.432991,1.867270,...,35.335209,49.607727,33.896358,19.865059,47.707191
2011-01-05,56.423058,32.220001,20.548147,40.182114,1.867270,...,35.289646,49.682713,33.622368,20.137701,47.742367
2011-01-06,54.567047,32.270000,20.435154,39.852474,1.852715,...,35.133995,49.403278,33.590519,19.622097,46.264721
2011-01-07,56.024700,32.040001,20.354973,40.333736,1.838162,...,36.132416,49.171574,33.393009,19.459620,46.255913
...,...,...,...,...,...,...,...,...,...,...,...
2015-12-24,136.800491,94.300003,24.968616,50.374283,3.333859,...,66.289261,76.529037,38.659355,31.603209,93.806572
2015-12-28,138.103104,94.199997,25.055744,50.292648,3.350689,...,65.902275,76.261749,38.896095,31.630283,93.204941
2015-12-29,139.616714,95.330002,25.283939,51.060093,3.396127,...,66.432251,76.827820,39.348686,31.941502,93.880936


,2011-03-15,2011-03-16,2011-03-17,2011-03-18,2011-03-21,...,2015-12-24,2015-12-28,2015-12-29,2015-12-30,2015-12-31
item_pair,,,,,,,,,,,
AAP & ADBE,0.168728,0.222818,0.144486,0.071317,-0.051968,...,-0.643238,-0.646528,-0.645340,-0.643469,-0.637273
AAP & AFL,-0.151145,-0.224601,-0.257909,-0.299028,-0.370857,...,0.256145,0.301352,0.336443,0.380367,0.427734
AAP & AIG,-0.376550,-0.472210,-0.469067,-0.471769,-0.409881,...,-0.028387,-0.000002,0.008289,0.020591,0.041623
AAP & AIV,-0.220231,-0.356251,-0.343942,-0.322663,-0.261431,...,0.139777,0.102072,0.056665,-0.000011,-0.059428
AAP & ALB,0.013067,-0.022910,-0.075200,-0.131819,-0.239324,...,0.134816,0.078493,0.027804,-0.012875,-0.046434
...,...,...,...,...,...,...,...,...,...,...,...
UPS & VZ,0.425982,0.481486,0.511456,0.548931,0.554568,...,-0.186211,-0.237711,-0.288640,-0.318487,-0.324075
UPS & ZBH,0.591623,0.565058,0.554903,0.544061,0.539139,...,0.058006,0.057754,0.053665,0.057196,0.064937
VTR & VZ,0.194474,0.256653,0.288954,0.339807,0.344996,...,0.296318,0.360381,0.453870,0.560238,0.611292


,2011-03-15,2011-03-16,2011-03-17,2011-03-18,2011-03-21,...,2015-12-24,2015-12-28,2015-12-29,2015-12-30,2015-12-31
item_pair,,,,,,,,,,,
AAP & ADBE,0.0,0.0,0.0,0.0,0.0,...,-1.0,-1.0,-1.0,-1.0,-1.0
AAP & AFL,0.0,0.0,0.0,0.0,-1.0,...,0.0,1.0,1.0,1.0,1.0
AAP & AIG,-1.0,-1.0,-1.0,-1.0,-1.0,...,0.0,0.0,0.0,0.0,0.0
AAP & AIV,0.0,-1.0,-1.0,-1.0,0.0,...,0.0,0.0,0.0,0.0,0.0
AAP & ALB,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
UPS & VZ,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,-1.0,-1.0
UPS & ZBH,1.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0
VTR & VZ,0.0,0.0,0.0,1.0,1.0,...,0.0,1.0,1.0,1.0,1.0


,corr_stl_period,corr_stl_resid,corr_stl_trend_std,corr_stl_trend_coef,corr_ser_mean,corr_ser_std
items,,,,,,
AAP & ADBE,2,4.393670,0.412886,0.000025,0.408259,0.414401
AAP & AFL,2,5.094379,0.517458,-0.000040,0.191143,0.518118
AAP & AIG,2,5.178324,0.468456,0.000033,0.269551,0.469781
AAP & AIV,2,4.889044,0.398367,0.000099,0.310793,0.399669
AAP & ALB,2,5.070536,0.465532,-0.000286,0.192075,0.466175
...,...,...,...,...,...,...
UPS & VZ,2,4.737550,0.432183,-0.000200,0.359899,0.433206
UPS & ZBH,2,3.943512,0.432655,-0.000292,0.461387,0.433141
VTR & VZ,2,4.801099,0.478726,-0.000418,0.302840,0.479150


time: 2.09 s (started: 2023-12-19 22:36:37 +08:00)


# Random pick trainset

In [4]:
num_items = 45
selected_seed = None
rand_seed = load_data_cfg()["RANDOM_SEEDS"][selected_seed] if selected_seed is not None else None

JPY_LOGGER.info("="*80)
JPY_LOGGER.info(f"num_items: {num_items}")
JPY_LOGGER.info(f"rand_seed: {rand_seed}")
JPY_LOGGER.info("="*80)

[ywt_jupyter.INFO] ================================================================================
[ywt_jupyter.INFO] num_items: 45
[ywt_jupyter.INFO] rand_seed: None
[ywt_jupyter.INFO] ================================================================================
time: 2.06 ms (started: 2023-12-19 22:36:56 +08:00)


In [5]:
final_pick_items = gen_random_items(all_items=dataset_df.columns.tolist(), ret_items_len=num_items, verbose=1, rand_seed=rand_seed)

JPY_LOGGER.info("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! final_pick_items !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
JPY_LOGGER.info(f"\n{final_pick_items}")
JPY_LOGGER.info("!"*100)

DEBUG    [utils.select_items_utils.gen_random_items] random seed: 10
DEBUG    [utils.select_items_utils.gen_random_items] len(ret_items):45
DEBUG    [utils.select_items_utils.gen_random_items] ret_items:
['AAP', 'ADBE', 'AFL', 'AIV', 'ALB', 'APD', 'ATVI', 'C', 'CI', 'CMG', 'CNP', 'COF', 'CRM', 'DISH', 'EQR', 'ES', 'EXR', 'FDX', 'FFIV', 'FL', 'GIS', 'GLW', 'HBI', 'IFF', 'INCY', 'IPGP', 'LEN', 'LMT', 'LRCX', 'MA', 'MAC', 'MCHP', 'MDLZ', 'MSCI', 'NSC', 'OKE', 'PKG', 'PPL', 'PSA', 'PVH', 'RJF', 'RRC', 'UDR', 'VZ', 'ZBH']
[ywt_jupyter.INFO] !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! final_pick_items !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
[ywt_jupyter.INFO] 
['AAP', 'ADBE', 'AFL', 'AIV', 'ALB', 'APD', 'ATVI', 'C', 'CI', 'CMG', 'CNP', 'COF', 'CRM', 'DISH', 'EQR', 'ES', 'EXR', 'FDX', 'FFIV', 'FL', 'GIS', 'GLW', 'HBI', 'IFF', 'INCY', 'IPGP', 'LEN', 'LMT', 'LRCX', 'MA', 'MAC', 'MCHP', 'MDLZ', 'MSCI', 'NSC', 'OKE', 'PKG', 'PPL', 'PSA', 'PVH', 'RJF', 'RRC', 'UDR', 'VZ', 'ZBH']
[ywt_jupyter.INFO]

# Correlation Series Property filtered trainset

In [ ]:
selected_corr_prop = "corr_mean"  # corr_mean|corr_std
selected_corr_prop_cond = "negative_corr_prop"
tmp_clique_dir = cliques_dir/f"{selected_corr_prop}/{selected_corr_prop_cond}"
fill_diag_val = 1
ret_items_len = 4
can_check_filtering_proc = True
corr_ser_std = corr_property_df.loc[::, "corr_ser_std"]
corr_ser_mean = corr_property_df.loc[::, "corr_ser_mean"]
selected_corr_prop_ser = corr_ser_mean if selected_corr_prop == "corr_mean" else corr_ser_std
item_names = tuple(dataset_df.columns)

In [ ]:
final_pick_items = gen_corr_prop_filtered_items(item_pairs_ser=selected_corr_prop_ser, corr_prop_cond=selected_corr_prop_cond,
                                                item_names=item_names, fill_diag_val=fill_diag_val, ret_items_len=ret_items_len,
                                                cliques_dir=tmp_clique_dir, can_check_filtering_proc=can_check_filtering_proc)

JPY_LOGGER.info("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! final_pick_items !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
JPY_LOGGER.info(f"{final_pick_items}")
JPY_LOGGER.info("!"*100)

# PCA and Clustering filtering on items | PCA and Clustering filtering on item_pairs

## Observe hyper parameters of PCA & Cluster model

### Observe big range hyper parameters of PCA & Cluster model

In [ ]:
filtering_on_items = False
filtering_on_item_pairs = True
assert filtering_on_items^filtering_on_item_pairs
if filtering_on_items:
    pca_input_data = dataset_df.T
    save_fig_dir = cluster_dir/"pca_hrchy/cluxter_with_items"
elif filtering_on_item_pairs:
    pca_input_data = corr_df
    save_fig_dir = cluster_dir/"pca_hrchy/cluxter_with_pairs"

obs_grid_pca_explanation_variance_thres = [1e-2]
pca_input_data_samples = pca_input_data.index
pca_input_data_featues = pca_input_data.columns
obs_grid_n_clusters = range(2, 50)
obs_linkage = "complete"
obs_cluster_metric = "euclidean"
save_fig_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
for obs_pca_explanation_variance_thres in obs_grid_pca_explanation_variance_thres:
    save_fig_path = save_fig_dir/f"{datetime.now().strftime('%Y%m%d%H%M%S')}_pca_thres({str(obs_pca_explanation_variance_thres).replace('.', '')})_silhouette_avg_vs_n_clusters"
    obs_reducted_data_df, obs_pri_components = calc_pca(data=pca_input_data, n_samples=len(pca_input_data_samples),
                                                        variance_thres=obs_pca_explanation_variance_thres, verbose=0)
    obs_cluster_conditions = {"n_samples": len(pca_input_data_samples), "n_features": len(obs_pri_components), "n_clusters_list": obs_grid_n_clusters,
                              "linkage": obs_linkage, "cluster_metric": obs_cluster_metric}
    obs_various_n_clusters_hrchy_cluster(data=obs_reducted_data_df, cluster_conds=obs_cluster_conditions, can_plot_each_cluster_info=False, save_fig_path=save_fig_path)

    JPY_LOGGER.info("="*100)
    JPY_LOGGER.info(f"pca_explanation_variance_thres:{obs_pca_explanation_variance_thres}")
    JPY_LOGGER.info(f"pca_input_data.shape:{pca_input_data.shape}, len(pca_input_data_samples):{len(pca_input_data_samples)}, len(pca_input_data_featues):{len(pca_input_data_featues)}")
    JPY_LOGGER.info(f"pca_input_data_samples[:3]:{pca_input_data_samples[:3]}")
    JPY_LOGGER.info(f"pca_input_data_featues[:3]:{pca_input_data_featues[:3]}")
    JPY_LOGGER.info(f"cluster_conditions: {obs_cluster_conditions}")
    JPY_LOGGER.info("="*100)

### Reduce range of hyper params of PCA and Clustering, and observe again

In [ ]:
obs_selected_pca_explanation_variance_thres = 1e-2
obs_selected_n_clusters_list = [2193]

In [ ]:
obs_reducted_data_df, obs_pri_components = calc_pca(data=pca_input_data, n_samples=len(pca_input_data_samples),
                                                    variance_thres=obs_selected_pca_explanation_variance_thres, verbose=0)
obs_cluster_conditions = {"n_samples": len(pca_input_data_samples), "n_features": len(obs_pri_components), "n_clusters_list": obs_selected_n_clusters_list,
                          "linkage": obs_linkage, "cluster_metric": obs_cluster_metric}
obs_various_n_clusters_hrchy_cluster(data=obs_reducted_data_df, cluster_conds=obs_cluster_conditions, can_plot_each_cluster_info=True)

JPY_LOGGER.info("="*100)
JPY_LOGGER.info(f"pca_explanation_variance_thres:{obs_selected_pca_explanation_variance_thres}")
JPY_LOGGER.info(f"pca_input_data.shape:{pca_input_data.shape}, len(pca_input_data_samples):{len(pca_input_data_samples)}, len(pca_input_data_featues):{len(pca_input_data_featues)}")
JPY_LOGGER.info(f"pca_input_data_samples[:3]:{pca_input_data_samples[:3]}")
JPY_LOGGER.info(f"pca_input_data_featues[:3]:{pca_input_data_featues[:3]}")
JPY_LOGGER.info(f"cluster_conditions: {obs_cluster_conditions}")
JPY_LOGGER.info("="*100)

## Select items

In [ ]:
pca_kwargs = {"n_samples":len(pca_input_data_samples), "pca_explanation_variance_thres":  1e-2}
linkage = "complete"
cluster_metric = "euclidean"
selected_n_clusters = 2193
cluster_kwargs = {"n_clusters": selected_n_clusters, "linkage": linkage, "cluster_metric": cluster_metric}

JPY_LOGGER.info("="*80)
JPY_LOGGER.info(f"pca_input_data.shape:{pca_input_data.shape}, len(pca_input_data_samples):{len(pca_input_data_samples)}, len(pca_input_data_featues):{len(pca_input_data_featues)}")
JPY_LOGGER.info(f"pca_input_data_samples[:3]:{pca_input_data_samples[:3]}")
JPY_LOGGER.info(f"pca_input_data_featues[:3]:{pca_input_data_featues[:3]}")
JPY_LOGGER.info(f"pca_kwargs:{pca_kwargs}")
JPY_LOGGER.info(f"cluster_kwargs:{cluster_kwargs}")
JPY_LOGGER.info("="*80)

In [ ]:
if filtering_on_items:
    pick_items_two_max_dist_clusters, final_pick_items= gen_pca_cluster_filtered_samples_two_max_dist_clusters(pca_input_data=pca_input_data, pca_kwargs=pca_kwargs, cluster_kwargs=cluster_kwargs, filter_on="items", verbose=1)
    assert random.choice(final_pick_items["items"]).count(" & ") == 0
    JPY_LOGGER.info("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! final_pick_items !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    JPY_LOGGER.info(f"pick_items_two_max_dist_clusters:\n{pformat(pick_items_two_max_dist_clusters, width=100)}")
    JPY_LOGGER.info(f"final_pick_items:\n{pformat(final_pick_items, width=100)}")
    JPY_LOGGER.info("!"*100)
elif filtering_on_item_pairs:
    pick_pairs_two_max_dist_clusters, final_pick_pairs = gen_pca_cluster_filtered_samples_two_max_dist_clusters(pca_input_data=pca_input_data, pca_kwargs=pca_kwargs, cluster_kwargs=cluster_kwargs, filter_on="pairs", verbose=1)
    assert random.choice(final_pick_pairs["pairs"]).count(" & ") == 1
    JPY_LOGGER.info("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! final_pick_pairs !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    JPY_LOGGER.info(f"pick_pairs_two_max_dist_clusters:\n{pformat(pick_pairs_two_max_dist_clusters, width=100)}")
    JPY_LOGGER.info(f"final_pick_pairs:\n{pformat(final_pick_pairs, width=100)}")
    JPY_LOGGER.info("!"*100)

## Select items from each cluster that use above settings

In [ ]:
num_selected_clusters = 10
num_samples_each_cluster = 1

JPY_LOGGER.info("="*80)
JPY_LOGGER.info(f"num_selected_clusters: {num_selected_clusters}, num_samples_each_cluster:{num_samples_each_cluster}")
JPY_LOGGER.info(f"pca_input_data.shape:{pca_input_data.shape}, len(pca_input_data_samples):{len(pca_input_data_samples)}, len(pca_input_data_featues):{len(pca_input_data_featues)}")
JPY_LOGGER.info(f"pca_input_data_samples[:3]:{pca_input_data_samples[:3]}")
JPY_LOGGER.info(f"pca_input_data_featues[:3]:{pca_input_data_featues[:3]}")
JPY_LOGGER.info(f"pca_kwargs:{pca_kwargs}")
JPY_LOGGER.info(f"cluster_kwargs:{cluster_kwargs}")
JPY_LOGGER.info("="*80)

In [ ]:
if filtering_on_items:
    pick_items_each_cluster, final_pick_items= gen_pca_cluster_filtered_samples_each_cluster(pca_input_data=pca_input_data, num_selected_clusters=num_selected_clusters, num_samples_each_cluster=num_samples_each_cluster,
                                                                                             pca_kwargs=pca_kwargs, cluster_kwargs=cluster_kwargs, filter_on="items")
    assert random.choice(final_pick_items["items"]).count(" & ") == 0
    JPY_LOGGER.info("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! final_pick_items !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    JPY_LOGGER.info(f"pick_items_each_cluster:\n{pformat(pick_items_each_cluster, width=100)}")
    JPY_LOGGER.info(f"final_pick_items:\n {pformat(final_pick_items, width=200)}")
    JPY_LOGGER.info("!"*100)
elif filtering_on_item_pairs:
    pick_pairs_each_cluster, final_pick_pairs= gen_pca_cluster_filtered_samples_each_cluster(pca_input_data=pca_input_data, num_selected_clusters=num_selected_clusters, num_samples_each_cluster=num_samples_each_cluster,
                                                                                             pca_kwargs=pca_kwargs, cluster_kwargs=cluster_kwargs, filter_on="pairs")
    assert random.choice(final_pick_pairs["pairs"]).count(" & ") == 1
    JPY_LOGGER.info("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! final_pick_pairs !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    JPY_LOGGER.info(f"pick_pairs_each_cluster:\n{pformat(pick_pairs_each_cluster, width=100)}")
    JPY_LOGGER.info(f"final_pick_pairs:\n {pformat(final_pick_pairs, width=1000)}")
    JPY_LOGGER.info("!"*100)


## Select samples with given specific sample

In [ ]:
designated_sample = "ADBE & ZBH"

JPY_LOGGER.info("="*80)
JPY_LOGGER.info(f"designated_sample: {designated_sample}")
JPY_LOGGER.info(f"pca_input_data.shape:{pca_input_data.shape}, len(pca_input_data_samples):{len(pca_input_data_samples)}, len(pca_input_data_featues):{len(pca_input_data_featues)}")
JPY_LOGGER.info(f"pca_input_data_samples[:3]:{pca_input_data_samples[:3]}")
JPY_LOGGER.info(f"pca_input_data_featues[:3]:{pca_input_data_featues[:3]}")
JPY_LOGGER.info(f"pca_kwargs:{pca_kwargs}")
JPY_LOGGER.info(f"cluster_kwargs:{cluster_kwargs}")
JPY_LOGGER.info("="*80)

In [ ]:
if filtering_on_items:
    final_pick_items = gen_pca_cluster_samples_with_given_sample(pca_input_data=pca_input_data, given_sample=designated_sample, pca_kwargs=pca_kwargs, cluster_kwargs=cluster_kwargs, filter_on="items")
    JPY_LOGGER.info("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! final_pick_items !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    JPY_LOGGER.info(f"final_pick_items:\n {pformat(final_pick_items, width=200)}")
    JPY_LOGGER.info("!"*100)
elif filtering_on_item_pairs:
    final_pick_pairs = gen_pca_cluster_samples_with_given_sample(pca_input_data=pca_input_data, given_sample=designated_sample, pca_kwargs=pca_kwargs, cluster_kwargs=cluster_kwargs, filter_on="pairs")
    JPY_LOGGER.info("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! final_pick_pairs !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    JPY_LOGGER.info(f"final_pick_pairs:\n {pformat(final_pick_pairs, width=1000)}")
    JPY_LOGGER.info("!"*100)

# ...